In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import statsmodels.api as sm
from sklearn import preprocessing
import warnings
from scipy import stats
from statsmodels.sandbox.regression.predstd import wls_prediction_std
warnings.filterwarnings('ignore')
color = sns.color_palette()

Read in data

In [4]:
train_score_feature = pd.read_csv('train_score_feature.csv')

In [9]:
# maybe deal with nan data

In [10]:
# maybe deal with outliers

Build features for selected continuous features.

In [107]:
continuous_variables = ['bathroomcnt', 'bedroomcnt', 'calculatedfinishedsquarefeet', 'structuretaxvaluedollarcnt', 'taxvaluedollarcnt', 'landtaxvaluedollarcnt', 'taxamount']

In [108]:
continous_feature = train_score_feature[continuous_variables]

In [152]:
# month feature


transactiondate    01
dtype: object

In [5]:
month_feature = pd.get_dummies(train_score_feature['transactiondate'].apply(lambda x: str(x).split('-')[1]))

In [7]:
month_feature.columns

Index(['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12'], dtype='object')

In [109]:
# fill na for each variable
continous_feature.isnull().sum()

bathroomcnt                       0
bedroomcnt                        0
calculatedfinishedsquarefeet    661
structuretaxvaluedollarcnt      380
taxvaluedollarcnt                 1
landtaxvaluedollarcnt             1
taxamount                         6
dtype: int64

In [110]:
# use taxvaluedollarcnt/ 73 to impute taxamount
index = continous_feature['taxvaluedollarcnt'].notnull() & continous_feature['taxamount'].isnull()
continous_feature.loc[index, 'taxamount'] = continous_feature[index]['taxvaluedollarcnt'] / 73

In [111]:
# use taxvaluedollarcnt - landtaxvaluedollarcnt to impute structuretaxvaluedollarcnt
index = continous_feature['taxvaluedollarcnt'].notnull() & continous_feature['landtaxvaluedollarcnt'].notnull() \
& continous_feature['structuretaxvaluedollarcnt'].isnull()
continous_feature.loc[index, 'structuretaxvaluedollarcnt'] = continous_feature[index]['taxvaluedollarcnt'] - \
continous_feature[index]['landtaxvaluedollarcnt']

In [121]:
# Use mean value to fill the rest
continous_feature = continous_feature.fillna(continous_feature.mean())

In [123]:
# maybe turn lot size to categorical

Build categorical features

In [14]:
# variables with all categories

In [65]:
full_categorical_variables = ['airconditioningtypeid', 'heatingorsystemtypeid', 'fireplacecnt', 'garagecarcnt', 
                             'propertylandusetypeid', 'regionidcounty', 'unitcnt', 'buildingqualitytypeid', 'taxdelinquencyflag']
binary_variables = ['garagetotalsqft']

In [68]:
full_categorical_feature = train_score_feature[full_categorical_variables + binary_variables].fillna(-1).copy()

In [69]:
for feature in binary_variables:
    full_categorical_feature[full_categorical_feature[feature] > 0][feature] = 1

In [70]:
full_categorical_one_hot_feature = pd.get_dummies(full_categorical_feature, columns=full_categorical_variables)
print(len(full_categorical_one_hot_feature.columns))

82


In [36]:
# Sample columne names: 'airconditioningtypeid_-1.0', 'airconditioningtypeid_1.0'

In [37]:
# variables with selected categories

In [46]:
selected_categorical_variables = ['propertyzoningdesc', 'regionidneighborhood', 'regionidzip']

In [47]:
selected_categorical_feature = train_score_feature[selected_categorical_variables].fillna(-1)

In [50]:
feature_names = ['propertyzoningdesc_CA&MUR&D*', 'propertyzoningdesc_RBR4YY', 'propertyzoningdesc_SDSF7500RP', 'propertyzoningdesc_LRR10000*', 'propertyzoningdesc_WVRPD12U-R', 'propertyzoningdesc_PDR16000*', 'propertyzoningdesc_LBCCA', 'propertyzoningdesc_SDSF7500*', 'propertyzoningdesc_LCR3800030', 'propertyzoningdesc_WHR172', 'propertyzoningdesc_SGR17500SI', 'propertyzoningdesc_WCR19450*', 'propertyzoningdesc_SLR3*', 'propertyzoningdesc_CVR1*', 'propertyzoningdesc_PDSP*', 'propertyzoningdesc_BFA1*', 'regionidneighborhood_416343.0', 'regionidneighborhood_764108.0', 'regionidneighborhood_764105.0', 'regionidneighborhood_763219.0', 'regionidneighborhood_275129.0', 'regionidneighborhood_762684.0', 'regionidneighborhood_275916.0', 'regionidneighborhood_417225.0', 'regionidneighborhood_763171.0', 'regionidneighborhood_275979.0', 'regionidneighborhood_265889.0', 'regionidneighborhood_762177.0', 'regionidneighborhood_417433.0', 'regionidneighborhood_623378.0', 'regionidneighborhood_268316.0', 'regionidneighborhood_763220.0', 'regionidneighborhood_268132.0', 'regionidneighborhood_275795.0', 'regionidneighborhood_275795.0', 'regionidzip_96943.0', 'regionidzip_96951.0', 'regionidzip_96951.0']

In [51]:
selected_categorical_one_hot_feature = pd.get_dummies(selected_categorical_feature, columns=selected_categorical_variables)[feature_names]

In [39]:
# two variable interactions (not added for now)

Modeling

In [71]:
from sklearn.ensemble import RandomForestRegressor
from sklearn import cross_validation

In [140]:
model = RandomForestRegressor(n_jobs=10, n_estimators=200, max_features='auto')

In [129]:
cross_validation.cross_val_score(model, continous_feature, train_score_feature['logerror'].values, cv=5)

array([-0.03731465, -0.06672267, -0.06536473, -0.05051295, -0.0628703 ])

In [130]:
combined_feature = pd.concat([continous_feature, full_categorical_one_hot_feature, selected_categorical_one_hot_feature], axis=1)

In [163]:
continous_feature_shuffled = continous_feature.sample(frac=1).reset_index(drop=True)
model = RandomForestRegressor(n_jobs=10, n_estimators=100, max_features='auto')
cross_validation.cross_val_score(model, continous_feature_shuffled, train_score_feature['logerror'].values, cv=5, scoring='mean_absolute_error')

array([-0.08179804, -0.07744923, -0.07502386, -0.0744488 , -0.07401688])

In [141]:
model = RandomForestRegressor(n_jobs=10, n_estimators=200, max_features='auto')
cross_validation.cross_val_score(model, combined_feature, train_score_feature['logerror'].values, cv=5, scoring='mean_absolute_error')

array([-0.08026961, -0.07574933, -0.07338571, -0.07185328, -0.07228359])

In [160]:
shuffled = month_feature.sample(frac=1).reset_index(drop=True)

In [159]:
model = RandomForestRegressor(n_jobs=10, n_estimators=400, max_features='auto')
cross_validation.cross_val_score(model, month_feature, train_score_feature['logerror'].values, cv=5, scoring='mean_absolute_error')

array([-0.07378388, -0.07915132, -0.06646209, -0.06892036, -0.08587292])

In [161]:
model = RandomForestRegressor(n_jobs=10, n_estimators=200, max_features='auto')
cross_validation.cross_val_score(model, shuffled, train_score_feature['logerror'].values, cv=5, scoring='mean_absolute_error')

array([-0.07395926, -0.06974778, -0.06706454, -0.06601702, -0.06595985])

In [1]:
month_feature

NameError: name 'month_feature' is not defined